Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Nicholas Kushnir"
COLLABORATORS = "Arpan Chauhan, Mann Patel"

In [ ]:
pandas, sqlite3

In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import csv
from datetime import datetime

In [2]:
#Loading the database



file_path = '/home/njk56/cs210/Course Project Proposal/BTC-cleaned.csv'
data = pd.read_csv(file_path)
print(data.columns)

data_cleaned = data.copy()
numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume']

for column in numeric_columns:
    data_cleaned[column] = data_cleaned[column].str.replace(',', '').astype(float)

db_path = 'bitcoin_data.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS bitcoin_data (
    Date TEXT PRIMARY KEY,
    Open REAL,
    High REAL,
    Low REAL,
    Close REAL,
    Volume REAL
)
''')

data_cleaned.to_sql('bitcoin_data', conn, if_exists='replace', index=False)

conn.commit()


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')


In [3]:
conn = sqlite3.connect(db_path)
query = "SELECT MIN(Date) as Earliest, MAX(Date) as Latest FROM bitcoin_data;"
date_range = pd.read_sql_query(query, conn)

print(date_range)



     Earliest      Latest
0  2014-09-17  2024-11-22


In [ ]:
#Bitcoin Over Lifetime
query = f"""
SELECT * 
FROM bitcoin_data;
"""
data = pd.read_sql_query(query, conn)


data['Date'] = pd.to_datetime(data['Date'])

plt.figure(figsize=(12, 6))
plt.plot(data['Date'], data['Close'], label='Close Price', color='blue', linewidth=2)

plt.title('Bitcoin Prices Over Its Lifetime', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Close Price (USD)', fontsize=12)
plt.grid(visible=True, linestyle='--', alpha=0.5)
plt.legend(fontsize=12)
plt.tight_layout()

plt.show()

In [5]:
#Covid price data

query = f"""
SELECT * 
FROM bitcoin_data 
WHERE Date >= '2019-12-01' AND Date <= '2022-05-01';
"""
covid_data = pd.read_sql_query(query, conn)

print(covid_data)

           Date      Open      High       Low     Close        Volume
0    2022-05-01  37713.27  38627.86  37585.79  38469.09  2.700276e+10
1    2022-04-30  38605.86  38771.21  37697.94  37714.88  2.389571e+10
2    2022-04-29  39768.62  39887.27  38235.54  38609.82  3.088299e+10
3    2022-04-28  39241.43  40269.46  38941.42  39773.83  3.390370e+10
4    2022-04-27  38120.30  39397.92  37997.31  39241.12  3.098102e+10
..          ...       ...       ...       ...       ...           ...
878  2019-12-05   7253.24   7743.43   7232.68   7448.31  1.881609e+10
879  2019-12-04   7320.13   7539.78   7170.92   7252.03  2.166424e+10
880  2019-12-03   7323.98   7418.86   7229.36   7320.15  1.479749e+10
881  2019-12-02   7424.04   7474.82   7233.40   7321.99  1.708204e+10
882  2019-12-01   7571.62   7571.62   7291.34   7424.29  1.872071e+10

[883 rows x 6 columns]


In [7]:
#Save csv

def fetch_data_by_date_range(start_date, end_date, output_file):
    conn = sqlite3.connect('bitcoin_data.db')
    c = conn.cursor()
    try:
        datetime.strptime(start_date, '%Y-%m-%d')
        datetime.strptime(end_date, '%Y-%m-%d')
    except ValueError:
        print("Incorrect date format. Please use YYYY-MM-DD.")
        return

    query = """SELECT date, open FROM bitcoin_data 
               WHERE date BETWEEN ? AND ?"""
    c.execute(query, (f"{start_date} 00:00:00", f"{end_date} 23:59:59")) 
    
    data = c.fetchall()
    

    if data:
        with open(output_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Date", "Price"])  
            for row in data:
                writer.writerow(row)

        print(f"Data from {start_date} to {end_date} has been written to {output_file}")
    else:
        print(f"No data found for the date range from {start_date} to {end_date}")
if __name__ == "__main__":
    start_date_input = '2019-12-01'
    end_date_input = '2022-05-01'
    output_file = 'covid.csv'
    fetch_data_by_date_range(start_date_input, end_date_input, output_file)

Data from 2019-12-01 to 2022-05-01 has been written to covid.csv


In [ ]:
#Plotting Covid price data for Bitcoin
covid_data['Date'] = pd.to_datetime(covid_data['Date'])

plt.figure(figsize=(12, 6))
plt.plot(covid_data['Date'], covid_data['Close'], label='Close Price', color='blue', linewidth=2)

plt.title('Bitcoin Prices Over Covid', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Close Price (USD)', fontsize=12)
plt.grid(visible=True, linestyle='--', alpha=0.5)
plt.legend(fontsize=12)
plt.tight_layout()

plt.show()

In [ ]:
# Get high and low from covid and compute percent return

start_date = '2019-12-01'
end_date = '2022-05-01'

highest_price = covid_data['High'].max()
lowest_price = covid_data['Low'].min()

print(f"The highest price in the period is: {highest_price}")
print(f"The lowest price in the period is: {lowest_price}")

covid_return = (highest_price-lowest_price)/lowest_price
print(f"Total return from {start_date} to {end_date}: {covid_return * 100:.2f}%")

In [ ]:
#Look at 2020 election
query = f"""
SELECT * 
FROM bitcoin_data 
WHERE Date >= '2019-12-01' AND Date <= '2021-04-01';
"""
election1_data = pd.read_sql_query(query, conn)

election1_data['Date'] = pd.to_datetime(election1_data['Date'])

plt.figure(figsize=(12, 6))
plt.plot(election1_data['Date'], election1_data['Close'], label='Close Price', color='blue', linewidth=2)

plt.title('Bitcoin Prices Over The 2020 Election', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Close Price (USD)', fontsize=12)
plt.grid(visible=True, linestyle='--', alpha=0.5)
plt.legend(fontsize=12)
plt.tight_layout()

plt.show()

In [9]:
#Save csv

def fetch_data_by_date_range(start_date, end_date, output_file):
    conn = sqlite3.connect('bitcoin_data.db')
    c = conn.cursor()
    try:
        datetime.strptime(start_date, '%Y-%m-%d')
        datetime.strptime(end_date, '%Y-%m-%d')
    except ValueError:
        print("Incorrect date format. Please use YYYY-MM-DD.")
        return

    query = """SELECT date, open FROM bitcoin_data 
               WHERE date BETWEEN ? AND ?"""
    c.execute(query, (f"{start_date} 00:00:00", f"{end_date} 23:59:59")) 
    
    data = c.fetchall()
    

    if data:
        with open(output_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Date", "Price"])  
            for row in data:
                writer.writerow(row)

        print(f"Data from {start_date} to {end_date} has been written to {output_file}")
    else:
        print(f"No data found for the date range from {start_date} to {end_date}")
if __name__ == "__main__":
    start_date_input = '2019-12-01'
    end_date_input = '2021-04-01'
    output_file = 'election2020.csv'
    fetch_data_by_date_range(start_date_input, end_date_input, output_file)

Data from 2019-12-01 to 2021-04-01 has been written to election2020.csv


In [ ]:
# Get high and low from 2020 election and compute percent return

start_date = '2019-12-01'
end_date = '2021-04-01'

highest_price = election1_data['High'].max()
lowest_price = election1_data['Low'].min()

print(f"The highest price in the period is: {highest_price}")
print(f"The lowest price in the period is: {lowest_price}")

election1_return = (highest_price-lowest_price)/lowest_price
print(f"Total return from {start_date} to {end_date}: {election1_return * 100:.2f}%")

In [ ]:
#2024 Election
query = f"""
SELECT * 
FROM bitcoin_data 
WHERE Date >= '2023-12-01'
"""
election2_data = pd.read_sql_query(query, conn)

election2_data['Date'] = pd.to_datetime(election2_data['Date'])

plt.figure(figsize=(12, 6))
plt.plot(election2_data['Date'], election2_data['Close'], label='Close Price', color='red', linewidth=2)

plt.title('Bitcoin Prices Over The 2024 Election', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Close Price (USD)', fontsize=12)
plt.grid(visible=True, linestyle='--', alpha=0.5)
plt.legend(fontsize=12)
plt.tight_layout()

# Show the chart
plt.show()

In [10]:
#Save csv

def fetch_data_by_date_range(start_date, end_date, output_file):
    conn = sqlite3.connect('bitcoin_data.db')
    c = conn.cursor()
    try:
        datetime.strptime(start_date, '%Y-%m-%d')
        datetime.strptime(end_date, '%Y-%m-%d')
    except ValueError:
        print("Incorrect date format. Please use YYYY-MM-DD.")
        return

    query = """SELECT date, open FROM bitcoin_data 
               WHERE date BETWEEN ? AND ?"""
    c.execute(query, (f"{start_date} 00:00:00", f"{end_date} 23:59:59")) 
    
    data = c.fetchall()
    

    if data:
        with open(output_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Date", "Price"])  
            for row in data:
                writer.writerow(row)

        print(f"Data from {start_date} to {end_date} has been written to {output_file}")
    else:
        print(f"No data found for the date range from {start_date} to {end_date}")
if __name__ == "__main__":
    start_date_input = '2023-12-01'
    end_date_input = '2024-11-15'
    output_file = 'election2024.csv'
    fetch_data_by_date_range(start_date_input, end_date_input, output_file)

Data from 2023-12-01 to 2024-11-15 has been written to election2024.csv


In [ ]:
# Get high and low from 2024 election and compute percent return

start_date = '2023-12-01'


highest_price = election2_data['High'].max()
lowest_price = election2_data['Low'].min()

print(f"The highest price in the period is: {highest_price}")
print(f"The lowest price in the period is: {lowest_price}")

election2_return = (highest_price-lowest_price)/lowest_price
print(f"Total return from {start_date} to present: {election2_return * 100:.2f}%")

In [ ]:

conn = sqlite3.connect('bitcoin_data.db')
query = "SELECT Date, Close FROM bitcoin_data ORDER BY Date ASC;"
data = pd.read_sql_query(query, conn)

data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
data = data['Close']


In [ ]:
duplicates = data.index.duplicated(keep=False)
if duplicates.any():
    print(data[duplicates])


In [ ]:
# Remove rows with NaT in the index
data = data[data.index.notna()]
print(data)
print(data.index.isna().sum())


In [ ]:
data = data.asfreq('D')

In [ ]:

model = ExponentialSmoothing(data, seasonal='add', trend='add', seasonal_periods=365)
hw_model = model.fit()

forecast = hw_model.forecast(steps=30)


In [ ]:

plt.figure(figsize=(10, 6))
plt.plot(data, label='Observed')
plt.plot(hw_model.fittedvalues, label='Fitted', color='green')
plt.plot(forecast, label='Forecast', color='red')
plt.legend()
plt.title('Holt-Winters Forecast')
plt.show()


In [ ]:
print(forecast)